<a href="https://colab.research.google.com/github/MaryDongsn/GNG-5125-clustering/blob/master/List_of_Movie_Final_project_content_based_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from math import sqrt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
###### Use the following code if you use google colab #####
from google.colab import drive 
drive.mount('/content/gdrive')
ratings=pd.read_csv('/content/gdrive/My Drive/gng5125 project/ratings.csv',sep=',', encoding='latin-1', usecols=['userId','movieId','rating','timestamp'])
movies = pd.read_csv('/content/gdrive/My Drive/gng5125 project/movies.csv', sep=',', encoding='latin-1', usecols=['movieId','title','genres'])

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:

# # Reading ratings file
# ratings = pd.read_csv('ratings.csv', sep=',', encoding='latin-1', usecols=['userId','movieId','rating','timestamp'])

# # Reading movies file
# movies = pd.read_csv('movies.csv', sep=',', encoding='latin-1', usecols=['movieId','title','genres'])

df_movies = movies 
df_ratings = ratings

In [5]:

#remove the year of the movie
df_movies.insert(2, "title_only",df_movies['title'].str.rsplit(" (").str[0])
print (df_movies['title']) 
df_movies['title_only'] = df_movies['title_only'].str.lower()
#df_movies['title'] = df_movies['title'].str.rsplit(" (").str[0]
print (df_movies['title_only'])

0                                Toy Story (1995)
1                                  Jumanji (1995)
2                         Grumpier Old Men (1995)
3                        Waiting to Exhale (1995)
4              Father of the Bride Part II (1995)
                          ...                    
9737    Black Butler: Book of the Atlantic (2017)
9738                 No Game No Life: Zero (2017)
9739                                 Flint (2017)
9740          Bungo Stray Dogs: Dead Apple (2018)
9741          Andrew Dice Clay: Dice Rules (1991)
Name: title, Length: 9742, dtype: object
0                                toy story
1                                  jumanji
2                         grumpier old men
3                        waiting to exhale
4              father of the bride part ii
                       ...                
9737    black butler: book of the atlantic
9738                 no game no life: zero
9739                                 flint
9740          bungo st

In [6]:
# Define a TF-IDF Vectorizer Object.
tfidf_movies_genres = TfidfVectorizer(token_pattern = '[a-zA-Z0-9\-]+')

#Replace NaN with an empty string
df_movies['genres'] = df_movies['genres'].replace(to_replace="(no genres listed)", value="")

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_movies_genres_matrix = tfidf_movies_genres.fit_transform(df_movies['genres'])

# Compute the cosine similarity matrix

cosine_sim_movies = linear_kernel(tfidf_movies_genres_matrix, tfidf_movies_genres_matrix)
# print(cosine_sim_movies)

In [7]:
def get_recommendations_based_on_genres(movie_title, cosine_sim_movies=cosine_sim_movies):
    """
    Calculates top 2 movies to recommend based on given movie titles genres. 
    :param movie_title: title of movie to be taken for base of recommendation
    :param cosine_sim_movies: cosine similarity between movies 
    :return: Titles of movies recommended to user
    """
    # Get the index of the movie that matches the title
    movie_title = movie_title.lower()
    idx_movie = df_movies.loc[df_movies['title_only'].isin([movie_title])]
    idx_movie = idx_movie.index

    
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores_movies = list(enumerate(cosine_sim_movies[idx_movie][0]))
    
    # Sort the movies based on the similarity scores
    sim_scores_movies = sorted(sim_scores_movies, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies 
    sim_scores_movies = sim_scores_movies[0:10]

    
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores_movies if not i[0] == idx_movie  ]
   
    
    # Return the top 1 most similar movies
    return df_movies['title'].iloc[movie_indices]
    
  

In [8]:
my_movie = input("Enter the name and the year of the movie, forexample, Toy Story (1995):")
recommend_to_watch = get_recommendations_based_on_genres(my_movie)
print ("Based on " + my_movie + ", " + "we recommend you to watch: ")
print (recommend_to_watch.to_string(index=False))


Enter the name and the year of the movie, forexample, Toy Story (1995):toy story
Based on toy story, we recommend you to watch: 
                                       Antz (1998)
                                Toy Story 2 (1999)
    Adventures of Rocky and Bullwinkle, The (2000)
                  Emperor's New Groove, The (2000)
                             Monsters, Inc. (2001)
                                  Wild, The (2006)
                            Shrek the Third (2007)
                    Tale of Despereaux, The (2008)
 Asterix and the Vikings (AstÃ©rix et les Vikin...


In [ ]:
# #get_top_recommendations_based_on_genres = 
# my_movie = input("Enter the name and the year of the movie, forexample, Toy Story (1995):")


# recommend_to_watch = get_recommendations_based_on_genres(my_movie)
# print ("Based on " + my_movie + ", " + "we recommend you to watch " + recommend_to_watch + ".")